## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-26-10-32-45 +0000,nyt,Zelensky Says He’ll Meet With Trump in the ‘Ne...,https://www.nytimes.com/2025/12/26/world/europ...
1,2025-12-26-10-30-00 +0000,wsj,Rolex Tries to Beat Watch Flippers at Their Ow...,https://www.wsj.com/finance/rolex-watch-second...
2,2025-12-26-10-30-00 +0000,wsj,"Customs Crackdown Leads to Blocked, Destroyed ...",https://www.wsj.com/economy/trade/customs-crac...
3,2025-12-26-10-30-00 +0000,wsj,Silver’s Runaway Rally Sweeps Up Amateur Inves...,https://www.wsj.com/finance/commodities-future...
4,2025-12-26-10-30-00 +0000,wsj,Tens of thousands of imports are being stopped...,https://www.wsj.com/economy/trade/customs-crac...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
183,christmas,34
4,trump,18
84,new,11
502,pope,7
503,leo,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2025-12-25-23-24-19 +0000,nypost,Trump launches ‘powerful and deadly’ Christmas...,https://nypost.com/2025/12/25/us-news/trump-la...,81
115,2025-12-25-18-46-06 +0000,nypost,"Pope Leo XIV calls for peace in Middle East, U...",https://nypost.com/2025/12/25/world-news/pope-...,76
151,2025-12-25-12-09-11 +0000,nyt,Kennedy Center’s Christmas Eve Jazz Show Cance...,https://www.nytimes.com/2025/12/24/us/kennedy-...,75
132,2025-12-25-16-21-18 +0000,bbc,Pope Leo urges 'courage' to end Ukraine war in...,https://www.bbc.com/news/articles/cm21728nwxlo...,74
43,2025-12-26-02-59-25 +0000,nypost,Jimmy Kimmel says ‘tyranny is booming’ as he r...,https://nypost.com/2025/12/25/media/jimmy-kimm...,72


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,81,2025-12-25-23-24-19 +0000,nypost,Trump launches ‘powerful and deadly’ Christmas...,https://nypost.com/2025/12/25/us-news/trump-la...
115,42,2025-12-25-18-46-06 +0000,nypost,"Pope Leo XIV calls for peace in Middle East, U...",https://nypost.com/2025/12/25/world-news/pope-...
144,33,2025-12-25-14-31-56 +0000,nypost,Thousands of new species being discovered ever...,https://nypost.com/2025/12/25/world-news/thous...
67,29,2025-12-25-23-59-19 +0000,latimes,Federal judge blocks ICE from arresting immigr...,https://www.latimes.com/california/story/2025-...
94,27,2025-12-25-21-01-09 +0000,nyt,"No Power, No Heat, No Water: Odesa’s Days of H...",https://www.nytimes.com/2025/12/25/world/europ...
131,26,2025-12-25-16-37-49 +0000,cbc,King Charles says unity in diversity will help...,https://www.cbc.ca/news/world/king-charles-chr...
129,25,2025-12-25-16-49-47 +0000,nypost,Trump fans blast musician who canceled holiday...,https://nypost.com/2025/12/25/us-news/trump-fa...
124,25,2025-12-25-17-20-17 +0000,nypost,Zelensky says Ukrainians just want Putin dead ...,https://nypost.com/2025/12/25/world-news/zelen...
17,22,2025-12-26-10-00-00 +0000,wapo,She spent the war planning to stay in Gaza. No...,https://www.washingtonpost.com/world/2025/12/2...
120,21,2025-12-25-17-58-37 +0000,nypost,Amazon Prime slammed for streaming ‘It’s a Won...,https://nypost.com/2025/12/25/business/amazon-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
